In [122]:
import rdkit
import copy
from rdkit import Chem 
from rdkit.Chem import Draw, Kekulize, SanitizeMol
from rdkit.Chem.rdmolops import CombineMols

In [173]:
def idx_annotate(x):
    """
    Taken from: https://iwatobipen.wordpress.com/2017/02/25/draw-molecule-with-atom-index-in-rdkit/
    
    Add label to each molecule's atom: "atom_name:atom_index"
    """
    mol = copy.deepcopy(x)
    for idx in range(mol.GetNumAtoms()):
        mol.GetAtomWithIdx(idx).SetProp(
            'molAtomMapNumber', 
            str(mol.GetAtomWithIdx(idx).GetIdx())
        )
    return mol

def mol(x) : 
    return Chem.MolFromSmiles(x)



def draw(x) : 
    if type(x) == str : x = mol(x)
    Draw.MolToImage(x).show()

In [174]:
purine = Chem.MolFromSmiles('C1=C2C(=NC=N1)N=CN2')
addition = Chem.MolFromSmiles('CNC[C@@H](C)O')

In [180]:
def auto_add(x, y) : 
    if type(x) == str : x = mol(x)
    if type(y) == str : y = mol(y)
    
    combo = CombineMols(x, y) 
    output = []

    for i in range(x.GetNumAtoms()) :
        for j in range(x.GetNumAtoms(), combo.GetNumAtoms()) :
            for b in [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE] :
                combo_editable = Chem.EditableMol(combo)
                combo_editable.AddBond(i, j, order=b)

                try : 
                    Chem.SanitizeMol(combo_editable.GetMol())
                    output.append(Chem.MolToSmiles(combo_editable.GetMol()))
                except :
                    print('Failed')
    return output

In [181]:
output = auto_add(purine, addition)

Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed

[21:55:26] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7
[21:55:26] Explicit valence for atom # 0 C, 6, is greater than permitted
[21:55:26] Explicit valence for atom # 10 N, 4, is greater than permitted
[21:55:26] Explicit valence for atom # 0 C, 6, is greater than permitted
[21:55:26] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7
[21:55:26] Explicit valence for atom # 0 C, 6, is greater than permitted
[21:55:26] Explicit valence for atom # 12 C, 5, is greater than permitted
[21:55:26] Explicit valence for atom # 12 C, 6, is greater than permitted
[21:55:26] Explicit valence for atom # 0 C, 6, is greater than permitted
[21:55:26] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7
[21:55:26] Explicit valence for atom # 0 C, 6, is greater than permitted
[21:55:26] Explicit valence for atom # 14 O, 3, is greater than permitted
[21:55:26] Explicit valence for atom # 0 C, 6, is greater than permitted
[21:55:26] Can't kekulize mol.  Unkekulized atoms: 0 2 3 4 5 6 7
[21:55

In [182]:
for i in output : 
    print(i)

C[C@@H](O)CNCc1ncnc2nc[nH]c12
C[C@@H](O)CN(C)c1ncnc2nc[nH]c12
CNC(c1ncnc2nc[nH]c12)[C@@H](C)O
CNC[C@H](O)Cc1ncnc2nc[nH]c12
CNC[C@@H](C)Oc1ncnc2nc[nH]c12
C[C@@H](O)CNCc1ncc2[nH]cnc2n1
C[C@@H](O)CN(C)c1ncc2[nH]cnc2n1
CNC(c1ncc2[nH]cnc2n1)[C@@H](C)O
CNC[C@H](O)Cc1ncc2[nH]cnc2n1
CNC[C@@H](C)Oc1ncc2[nH]cnc2n1
C[C@@H](O)CNCc1nc2ncncc2[nH]1
C[C@@H](O)CN(C)c1nc2ncncc2[nH]1
CNC(c1nc2ncncc2[nH]1)[C@@H](C)O
CNC[C@H](O)Cc1nc2ncncc2[nH]1
CNC[C@@H](C)Oc1nc2ncncc2[nH]1
